In [1]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import pandas as pd
import yaml

In [2]:
%run ./src/database.py
%run ./src/util.py
# %run ./src/data_preprocessing.py

In [28]:
YAML_FILEPATHNAME = "./config.yaml"
PRE_CRUISE_DB = 0
POST_CRUISE_DB = 1

# Preprocessing

In [29]:
# Read YAML file
data_path = read_yaml(YAML_FILEPATHNAME)

In [34]:
# Read Pre_cruise Data
ds_pre_cruise = Database(DATA_PATH)
df_pre_cruise = ds_pre_cruise.db_read(yaml_data['databases'][PRE_CRUISE_DB])

In [35]:
# Read Post_cruise Data
ds_post_cruise = Database(DATA_PATH)
df_post_cruise = ds_post_cruise.db_read(yaml_data['databases'][POST_CRUISE_DB])

In [52]:


# Merge Pre_cruise and Post_cruise data
df_cruise = merge_dataframe (df_pre_cruise, df_post_cruise)

In [39]:
df_cruise.shape

(130875, 25)

In [ ]:
### There are 130875 rows and 25 columns

In [48]:
df_cruise.head(5)

,Gender,Date of Birth,Source of Traffic,Onboard Wifi Service,Embarkation/Disembarkation time convenient,Ease of Online booking,Gate location,Logging,Onboard Dining Service,Online Check-in,...,Onboard Service,Cleanliness,Ext_Intcode_x,Cruise Name,Ticket Type,Cruise Distance,Ext_Intcode_y,WiFi,Dining,Entertainment
index,,,,,,,,,,,,,,,,,,,,,
0,None,05/10/1973,Direct - Company Website,A little important,3.0,5.0,3.0,01/01/2023 0:00,Very important,2.0,...,2.0,3.0,LB446RWOOZI,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
1,Female,None,Indirect - Social Media,Not at all important,4.0,1.0,NaN,01/01/2023 0:01,Very important,NaN,...,4.0,4.0,LB138HKBECM,Blastoise,Deluxe,672 KM,LB138HKBECM,NaN,0,1.0
2,Female,22/07/1998,Indirect - Search Engine,None,3.0,0.0,5.0,01/01/2023 0:02,None,NaN,...,3.0,NaN,BL713UHBAAN,IAPRAS,Deluxe,1167 KM,BL713UHBAAN,NaN,0,0.0
3,Female,01/05/1970,Direct - Company Website,Very important,4.0,4.0,4.0,01/01/2023 0:05,Somewhat important,4.0,...,2.0,4.0,LB243DMKCFL,Lapras,Deluxe,280 KM,LB243DMKCFL,NaN,0,1.0
4,Male,07/01/1960,Direct - Company Website,Somewhat important,4.0,2.0,NaN,01/01/2023 0:06,Not at all important,2.0,...,2.0,NaN,LB218CFLOBS,Lapras,Standard,1145 Miles,LB218CFLOBS,NaN,1,NaN


In [53]:
# Split Source of Traffic into Source and Traffic
split_column(df_cruise, 'Source of Traffic', ['Source', 'Traffic'], ' - ')

In [55]:
# Split Source of Traffic into Source and Traffic
split_column(df_cruise, 'Cruise Distance', ['Distance', 'Dist_Matrix'], ' ')

# Feature Engineering

# Model Building

# Model Evaluation